<a href="https://colab.research.google.com/github/DSF-Kotaro/blog/blob/main/%E5%9B%B3%E8%A7%A3%E9%80%9F%E7%BF%92DeepLearning_021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

!apt-get update -qq && apt-get install -qq libfluidsynth1 build-essential libasound2-dev libjack-dev
!pip install -U magenta==0.5.0 pyfluidsynth

# Hack to allow python to pick up the newly-installed fluidsynth lib.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

# Download Salamander piano SoundFont.
# Samples by Alexander Holm: https://archive.org/details/SalamanderGrandPianoV3
# Converted to sf2 by John Nebauer: https://sites.google.com/site/soundfonts4u
!gsutil -m cp gs://download.magenta.tensorflow.org/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /tmp/

TensorFlow 1.x selected.
Selecting previously unselected package libfluidsynth1:amd64.
(Reading database ... 160975 files and directories currently installed.)
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 1.4MB 12.6MB/s 
     |████████████████████████████████| 7.5MB 39.2MB/s 
     |████████████████████████████████| 2.3MB 36.4MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 1.1MB 48.6MB/s 
     |████████████████████████████████| 61kB 5.6MB/s 
     |████████████████████████████████| 3.9MB 44.3MB/s 
     |████████████████████████████████| 20.5MB 1.4MB/s 
     |████████████████████████████████| 51kB 3.9MB/s 
     |█████████████████

Copying gs://download.magenta.tensorflow.org/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2...
\ [1/1 files][591.9 MiB/591.9 MiB] 100% Done                                    
Operation completed over 1 objects/591.9 MiB.                                    


In [2]:
import os
from magenta.models.performance_rnn import performance_sequence_generator
from magenta.protobuf import generator_pb2
from magenta.protobuf import music_pb2
import magenta.music as mm
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

W0316 00:17:07.997973 140064427456384 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/magenta/pipelines/statistics.py:131: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

W0316 00:17:11.178529 140064427456384 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/magenta/music/note_sequence_io.py:60: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W0316 00:17:11.185739 140064427456384 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
BUNDLE_DIR = '/tmp/'
MODEL_NAME = 'performance_with_dynamics'
BUNDLE_NAME = MODEL_NAME + '.mag'

mm.notebook_utils.download_bundle(BUNDLE_NAME, BUNDLE_DIR)

W0316 00:17:12.998403 140064427456384 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/magenta/music/notebook_utils.py:196: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.



In [4]:
bundle = mm.sequence_generator_bundle.read_bundle_file(os.path.join(BUNDLE_DIR, BUNDLE_NAME))
generator_map = performance_sequence_generator.get_generator_map()
generator = generator_map[MODEL_NAME](checkpoint=None, bundle=bundle)
generator.initialize()
generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = 1.0  # Higher is more random; 1.0 is default. 
generate_section = generator_options.generate_sections.add(start_time=0, end_time=30)
sequence = generator.generate(music_pb2.NoteSequence(), generator_options)

# Play and view this masterpiece.
mm.plot_sequence(sequence)
mm.play_sequence(sequence, mm.midi_synth.fluidsynth,
                 sf2_path='/tmp/Yamaha-C5-Salamander-JNv5.1.sf2')

W0316 00:17:25.344769 140064427456384 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/magenta/music/sequence_generator_bundle.py:30: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.

W0316 00:17:25.406018 140064427456384 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/magenta/music/model.py:69: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0316 00:17:25.521008 140064427456384 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/magenta/music/model.py:70: The name tf.train.import_meta_graph is deprecated. Please use tf.compat.v1.train.import_meta_graph instead.

W0316 00:17:25.642663 140064427456384 meta_graph.py:887] The saved meta_graph is possibly from an older release:
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
W0316 00:17:25.787916 140064427456384 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/magenta